In [1]:
%pip install azure-eventhub

     |████████████████████████████████| 118 kB 13.2 MB/s 
     |████████████████████████████████| 179 kB 72.7 MB/s 
     |████████████████████████████████| 3.1 MB 28.6 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
arviz 0.11.4 requires typing-extensions<4,>=3.7.4.3, but you have typing-extensions 4.1.1 which is incompatible.


In [2]:
from azure.eventhub import EventHubConsumerClient
from multiprocessing import Queue
from datetime import datetime
import logging

In [3]:
CONSUMER_GROUP = '$default'
D2C_EVENTHUB_NAME = 'imreceiverinput' #has 2 partitions
logger = logging.getLogger("azure.eventhub")
logging.basicConfig(level=logging.WARN)

# In a live environment, this should be loaded from an environment variable, not in code
CONN_STR='Endpoint=sb://simrioteventhubns.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=Gl0MbGOeYzhkYo7+/aLTNrl1/hSJSro/U+Z4VmdctTU='


In [4]:
# Utilities
def UTC_now_string():
    return( datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f") )

# Consumer class
A simple class to consume Azure Event Hub data

In [5]:

class EventConsumer:
    _client=None
    _conn_string=None
    _consumer_group=None
    _eventhub_name=None
    _queue=None
    _recv_task=None
        
    def __init__(self, connection_string, consumer_group, eventhub_name, queue):
        self._conn_string=connection_string
        self._consumer_group=consumer_group
        self._eventhub_name=eventhub_name
        self._queue=queue
    
    
    def on_event(self,partition_context, event):
        logger.info("Received event from partition {a} at {b}".format(a=partition_context.partition_id, b=UTC_now_string()) )
        print(event.body_as_str())
        partition_context.update_checkpoint(event)
        try:
            self._queue.put( event.body_as_str() )
        except:
            logger.error('An error occurred')

    def on_error(self, partition_context, exception):
        logger.error(exception)
    
    def create(self):
        self._client = EventHubConsumerClient.from_connection_string(
                self._conn_string, 
                self._consumer_group, 
                eventhub_name=self._eventhub_name,
                retry_total=2,
                logging_enable=True,
                on_error=self.on_error)
    
    def start(self):
        with self._client:
            self._client.receive(on_event=self.on_event, 
                                 starting_position=-1)
    

In [6]:
d2c_q = Queue()
d2c_consumer=EventConsumer(CONN_STR, CONSUMER_GROUP, D2C_EVENTHUB_NAME, d2c_q)
d2c_consumer.create()

## Start the Consumer
Click the stop button on the cell to shut down

In [ ]:
d2c_consumer.start()

## Function Version


In [ ]:
# Event Hub D2C CONSUMER
###########################################################
def d2c_consumer(q):
    def on_event(partition_context, event):
        logger.info("D2C Received event from partition {a} at {b}".format(a=partition_context.partition_id, b=UTC_now_string()) )
        #logger.info(event.body_as_str())
        print( event.body_as_str() )
        partition_context.update_checkpoint(event)
        try:
            q.put( event.body_as_str() )
        except:
            pass

    client = EventHubConsumerClient.from_connection_string(
                CONN_STR, CONSUMER_GROUP, 
                eventhub_name=D2C_EVENTHUB_NAME, logging_enable=True)
    with client:
        client.receive(
            on_event=on_event,
            starting_position="-1",  # "-1" is from the beginning of the partition.
        )


In [ ]:
# Run the consumer

## Q for receiving new DEVICE DATA
d2c_q = Queue()
d2c_consumer(d2c_q,)
